# Generate Simple Solution Phases with Ideal mixing

This notebook generates uses the SimpleSoln class of coder and SymPy to generate
models for ideal mixing in binary solution phases

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import hashlib
import time
sym.init_printing()

Required ENKI packages (using ThermoCodegen version of the coder package)

In [ ]:
from thermocodegen.coder import coder

Get the working version of ThermoCodegen

In [ ]:
import thermocodegen as tcg
tcg_version = tcg.__version__
print('Using ThermoCodegen v{}'.format(tcg_version))

and set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath(os.curdir)
SPUD_DIR = HOME_DIR+'/../phases'

try:
    os.mkdir(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference = 'Thermocodegen-v{}/examples/Systems/fo_fa/notebooks/Generate_phases.ipynb'.format(tcg_version)

## Ideal Solution properties
There are Two terms:
- Terms describing standard state contributions
- Terms describing the configurational entropy of solution

Assumptions:
- There are $c$ components in the system
- There are as many endmember species, $s$, as there are components
- The configurational entropy is described as a simple $x_i log(x_i)$ sum

This notebook illustrates a two component solution

In [ ]:
c = 2

## Create a simple solution model
... with the specified number of endmember thermodynamic components

In [ ]:
model = coder.SimpleSolnModel.from_type(nc=c)

### Retrieve primary compositional variables
- $n$ is a vector of mole numbers of each component  
- $n_T$ is the total number of moles in the solution

In [ ]:
n = model.n
nT = model.nT

### and construct a derived mole fraction variable
- $X$ is a vector of mole fractions of components in the system

In [ ]:
X = n/nT
n, nT, X

### Retrieve the temperature, pressure, and standard state chemical potentials
- $T$ is temperature in $K$
- $P$ is pressure in $bars$
- $\mu$ in Joules

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
mu = model.mu
T,P,mu

Check the model dictionary

In [ ]:
model.model_dict

### Define the standard state contribution to solution properties

$$
    G_{ss} = \sum_k n_k\mu_k(T,P)
$$

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

### Define configurational entropy and configurational Gibbs free energy

In [ ]:
S_config,R = sym.symbols('S_config R')
S_config = 0
for i in range(0,c):
    S_config += X[i]*sym.log(X[i])
S_config *= -R*nT
S_config

In [ ]:
G_config = sym.simplify(-T*S_config)
G_config

In [ ]:
param_string=['R']
units = ['J/K/mol']
params = coder.set_coder_params(param_string,units)
params

### Define the Gibbs free energy of solution

In [ ]:
G = G_ss + G_config 
G

### Add the Gibbs free energy of solution to the model

In [ ]:
model.add_potential_to_model('G',G,params)

### let's inspect the dictionary and unset parameters

In [ ]:
model.model_dict

In [ ]:
values_dict = model.get_values()
values_dict

## Define Parameters of a Fo-Fa ideal Solution
Components
1. forsterite, ${\rm{Mg_2SiO_4}}$
2. fayalite, ${\rm{Fe_2SiO_4}}$

* assign a formula string for code generation  
    * here the solution formula is Mg$_x$Fe$_{2-x}$SiO$_4$ where $x\in[0,2]$
* assign a conversion string to map element concentrations to moles of end members
    * here endmember 0 is Forsterite, ${\rm{Mg_2SiO_4}}$ so 1 mole of Forsterite will be 1/2 the number of Mg atoms
    * likewise for endmember 1, Fayalite,  1 mole of Fa will be 0.5* the number of Fe atoms whic gives the conversion_string for moles of endmember 0, and 1 as 
    
    `conversion_string=['[0]=0.5*[Mg]', '[1]=0.5*[Fe]']`


In [ ]:
values_dict.update(dict(formula_string='Mg[Mg]Fe[Fe]SiO4',
                        conversion_string=['[0]=0.5*[Mg]', '[1]=0.5*[Fe]'],
                        test_string = ['[0] > 0.0', '[1] > 0.0'],
                        R=8.31446261815324))
values_dict

### Add additional parameters for solid phase Olivine and liquid phase Liquid

In [ ]:
Ol_dict = dict(name='Olivine',abbrev='Ol',
                        reference=reference,
                        endmembers = ['Forsterite_berman', 'Fayalite_berman'])
Lq_dict = dict(name='Liquid',abbrev='Liq',
                        reference=reference,
                        endmembers = ['Forsterite_xmelts', 'Fayalite_xmelts'])

## Generate Spud XML files

and write them to the Spud endmember directory

In [ ]:
for d in [ Ol_dict, Lq_dict]:
    values_dict.update(d)
    model.set_values(values_dict)
    file = model.to_xml(path=SPUD_DIR)
    print(file)